#**Ejemplo detección de rostros usando webcam**

Bootcamp: Mi primera inteligencia artificial

Prof. Carlos Toro, carlos.toro.ing@gmail.com,  2023

Capturaremos una imagen desde la webcam y ejecutaremos detección de rostro sobre esta imagen capturada. La predicción de rostros en la imagen adquirida usando el clasificador en Cascada Haar pre-entrenado y disponible en OpenCV.

**Detección de Rostros:**

La siguiente función para capturar una imagen desde la webcam usanco google colab la proporciona la misma plataforma, en código ejecutado de forma local en sus PCs, simplemente usar las disponibles por OpenCV (mucho más simple, usar `VideoCapture()`)

Previo: Ejecutar las siguientes funciones de ayuda para capturar y formatear imágenes desde la webcam en Google Colab, utilidades proporcionadas por colab en ícono de "fragmentos de código"

In [1]:
#@title Captura una imagen desde webcam: take_photo()
from IPython.display import display, Javascript, Image
from google.colab.output import eval_js
from base64 import b64decode , b64encode
import matplotlib.pyplot as plt
import cv2, matplotlib
import numpy as np

def take_photo(filename='photo.jpg', quality=0.8): # por defecto, la imagen capturada se llamará photo.jpg
  js = Javascript('''
    async function takePhoto(quality) {
      const div = document.createElement('div');
      const capture = document.createElement('button');
      capture.textContent = 'Capture';
      div.appendChild(capture);

      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      // Wait for Capture to be clicked.
      await new Promise((resolve) => capture.onclick = resolve);

      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getVideoTracks()[0].stop();
      div.remove();
      return canvas.toDataURL('image/jpeg', quality);
    }
    ''')
  display(js)

  # get photo data
  data = eval_js('takePhoto({})'.format(quality))
  # get OpenCV format image
  img = js_to_image(data)
  #print(gray.shape)
  # save image
  cv2.imwrite(filename, img)

  return filename

  #function to convert the JavaScript object into an OpenCV image
def js_to_image(js_reply):
  """
  Params:
          js_reply: JavaScript object containing image from webcam
  Returns:
          img: OpenCV BGR image
  """
  # decode base64 image
  image_bytes = b64decode(js_reply.split(',')[1])
  # convert bytes to numpy array
  jpg_as_np = np.frombuffer(image_bytes, dtype=np.uint8)
  # decode numpy array into OpenCV BGR image
  img = cv2.imdecode(jpg_as_np, flags=1)

  return img

Código para detectar rostros en una imagen capturada con la webcamm:

In [ ]:
# Inicializamos el detector de rostros
face_cascade = cv2.CascadeClassifier(cv2.samples.findFile(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml'))

# realicemos una predicción sobre la imagen capturada
try:
  filename = take_photo('photo.jpg')#capturamos imagen
  img      = cv2.imread(filename)

# Obtener las coordenadas de los rostros detectados con detector Cascada Haar
  faces = face_cascade.detectMultiScale(img)
# dibujar caja rectangular en rostro detectado
  for (x,y,w,h) in faces:
      img = cv2.rectangle(img,(x,y),(x+w,y+h),(255,0,0),2)

  # Mostrar la imagen capturada y con los rectángulos que corresponda.
  plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
  plt.show
except Exception as err:
  # except se ejecuta solo si se producen errores en try:
  # útil, yaque a veces al trabajar con dispositvos como webcam,
  # se puede desconectar o generar un error al querer conectar.
  print(str(err))